In [23]:
## first time series test

# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 
import matplotlib.image as mplimg

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [24]:
from PIL import Image, ImageDraw, ImageFont
import os
import cv2
import numpy as np

In [27]:
# load the data

WIDTH=18158
nlines = 7932
SCALE = 1.0
EXP = 0.35

RMLI1 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/c20200927.rmli.tif'
RMLI2 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/c20201113.rmli.tif'
RMLI1 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201116.pwr.ras'
RMLI2 = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/x20201219.pwr.ras'

RMLI1_bin = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201116.rmli'
RMLI2_bin = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201127.rmli'
RMLI12_bin = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201116_x20201127_diff.rmli'

RMLI1_ras = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201116.rmli.ras'
RMLI2_ras = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201127.rmli.ras'
RMLI3_ras = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201208.rmli.ras'
RMLI4_ras = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201219.rmli.ras'
RMLI5_ras = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20211023.rmli.ras'
RMLI12_ras = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/TSX/staring_134/RMLI_4_gif/RMLI_diffs/x20201116_x20201127_diff.rmli.ras'



RMLI_ras = ['/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200910.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200919.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200926.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20200927.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201005.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201012.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201013.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201113.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201114.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201122.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201224.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20201231.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20210101.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20210109.rmli.ras',
        '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/CSK/dsc1/RMLIs/rmlis_4_gif/cropped_rmli_full/c20211101.rmli.ras']



In [57]:
# calculate difference
Image.MAX_IMAGE_PIXELS = WIDTH*nlines

image1 = np.asarray(Image.open(RMLI_ras[0]))
image2 = np.asarray(Image.open(RMLI_ras[13]))
image3 = np.asarray(Image.open(RMLI_ras[2]))
# image4 = np.asarray(Image.open(RMLI4_ras))
# image5 = np.asarray(Image.open(RMLI5_ras))
image_diff = image2 - image1
print(image_diff)

print(np.min(image1), np.max(image1))
print(np.min(image2), np.max(image2))
print(np.min(image_diff), np.max(image_diff))



[[106 215  18 ... 248 232  10]
 [ 46  63  47 ...  12  32  26]
 [ 11  45  35 ...   2 225 251]
 ...
 [  0 159 222 ...  70 216 220]
 [186  65 165 ... 174 135  58]
 [ 11 201  90 ... 186 185 168]]
0 255
0 255
0 255


In [58]:
# normalise to 0 - 1
def normalise_arr(arr):
    """Normalises N-D numpy array to values between 0 and 1 where the minimum value 
       in the original array becomes 0 and the maximum value becomes 1.

    Args:
        arr (Numpy array): ND numeric array
    """
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

RMLI1_arr_norm = normalise_arr(image1)
RMLI2_arr_norm = normalise_arr(image2)
# RMLI3_arr_norm = normalise_arr(image3)
RMLI13_arr_norm = normalise_arr(image_diff)

In [59]:
# add channels to image
# stack images
# rgb_image = np.dstack([RMLI1_arr_norm,
#                        RMLI3_arr_norm,
#                        RMLI13_arr_norm])

rgb_image = np.dstack([RMLI1_arr_norm,
                       RMLI2_arr_norm,
                       RMLI13_arr_norm])
print(np.shape(rgb_image))

(8000, 8000, 3)


In [60]:
# display image
%matplotlib osx
plt.close('all')
fig, ax = plt.subplots(1,1)

ax.imshow(np.fliplr(np.rot90(rgb_image,3)))

In [8]:
# get binary data into array
WIDTH=18158

SCALE = 1.0
EXP = 0.35
# get latitude and longitude data from file
RMLI1_vec = np.fromfile(RMLI1_bin, dtype='>f', count=-1)
RMLI2_vec = np.fromfile(RMLI2_bin, dtype='>f', count=-1)
RMLI12_vec = np.fromfile(RMLI12_bin, dtype='>f', count=-1)
# get number of lines in lat lon files
nlines = int(RMLI1_vec.shape[0]/WIDTH)
# reshape tomatrix for index referencing
RMLI1_arr = np.reshape(RMLI1_vec,(nlines,WIDTH))
RMLI2_arr = np.reshape(RMLI2_vec,(nlines,WIDTH))
RMLI12_arr = np.reshape(RMLI12_vec,(nlines,WIDTH))



In [12]:
# normalise to 0 - 1



def normalise_arr(arr):
    """Normalises N-D numpy array to values between 0 and 1 where the minimum value in the original array becomes 0 and the maximum value becomes 1.

    Args:
        arr (Numpy array): ND numeric array
    """
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

RMLI1_arr_norm = normalise_arr(RMLI1_arr)
RMLI2_arr_norm = normalise_arr(RMLI2_arr)
RMLI12_arr_norm = normalise_arr(RMLI2_arr-RMLI1_arr)



In [13]:
# add channels to image
# stack images
rgb_image = np.dstack([RMLI1_arr_norm,
                       RMLI12_arr_norm,
                       RMLI2_arr_norm])

In [14]:
# display image
%matplotlib osx
plt.close('all')
fig, ax = plt.subplots(1,1)

ax.imshow(rgb_image)

In [17]:
# calculate difference
Image.MAX_IMAGE_PIXELS = WIDTH*nlines

image1 = np.asarray(Image.open(RMLI1))
image2 = np.asarray(Image.open(RMLI2))
image_diff = image2 - image1
print(image_diff)

print(np.min(image1), np.max(image1))
print(np.min(image2), np.max(image2))
print(np.min(image_diff), np.max(image_diff))


[[203  99 112 ...  61 148 152]
 [121  71   0 ... 105 189 195]
 [ 35 103   0 ... 117 116 123]
 ...
 [137  84  93 ...  91 109 161]
 [125  93 129 ...  88 102 119]
 [124  84 203 ...  93 228 153]]
0 255
0 255
0 255


In [18]:
# stack images
rgb_image = np.dstack([image1,image_diff,image2])

In [19]:
# display image
%matplotlib osx
fig, ax = plt.subplots(1,1)

ax.imshow(rgb_image)

1   HIToolbox                           0x00007ff8141cf726 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 102
2   HIToolbox                           0x00007ff8141cf2b8 _ZN15MenuBarInstance14EnableAutoShowEv + 52
3   HIToolbox                           0x00007ff81413ecd7 _ZN15MenuBarInstance21UpdateAggregateUIModeE21MenuBarAnimationStylehhh + 1113
4   HIToolbox                           0x00007ff8141cf173 _ZN15MenuBarInstance19SetFullScreenUIModeEjj + 175
5   AppKit                              0x00007ff80dc9c287 -[NSApplication _setPresentationOptions:instance:flags:] + 1145
6   AppKit                              0x00007ff80daf1055 -[NSApplication _updateFullScreenPresentationOptionsForInstance:] + 582
7   AppKit                              0x00007ff80e53f7f0 -[_NSFullScreenSpace(PresentationInstance) activateFullScreenPresentationOptions] + 207
8   AppKit                              0x00007ff80e3925d8 -[_NSEnterFullScreenTransitionController _doSucceededToEnterFullScreen] + 721
